# Admission/Transfer/Discharge table with Trajectories

We want to look into the cases that have a large difference between the time0 predictions and time24 predictions.


We'll look at: the complete `1_4_cohort` here

- pat services
- pat lv of care
- patient trajectories from admission until 24hrs

Subgroups of the entire cohort will be analyzed in another notebook

In [185]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [186]:
# view data frame

# adjust this to hide ID columns for posting to github
forrepo=False

def view_df(df):
    hidecols = []
    if forrepo:
        hidecols=['anon_id', 'pat_enc_csn_id_coded', 'inpatient_data_id_coded'] # these are hidden columns
        
    return(df.drop(hidecols, axis=1, errors='ignore').head())

# Data

Load in the data

In [187]:
# read in data files
datadir = "../../DataTD/"
resultsdir = "../../OutputTD/3_models/1_4_cohort/"

savedir = "../../OutputTD/5_results_analysis/"

adt_file = datadir + "cohort_1_3_adt.csv"
results_file = resultsdir + "1_4_cohort_test_results.csv"
cohort_file = "../../OutputTD/1_cohort/1_4_cohort.csv"


full_adt = pd.read_csv(adt_file)
results = pd.read_csv(results_file)
cohort = pd.read_csv(cohort_file)

/home/eulalio/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [188]:
view_df(results)

cohort.pat_enc_csn_id_coded.nunique()

43980

# Bring in the ADT Table

We can use the ADT table to look at 

- pat_class
- pat_lv_of_care

In [189]:
# filter adt table down to test cohort
adt = full_adt[full_adt.pat_enc_csn_id_coded.isin(cohort.pat_enc_csn_id_coded)]

# these should match - it does!
print(adt.pat_enc_csn_id_coded.nunique())
print(adt.pat_enc_csn_id_coded.nunique())

view_df(adt)

43980
43980


,anon_id,pat_enc_csn_id_coded,effective_time_jittered_utc,seq_num_in_enc,pat_class,base_pat_class_c,pat_lvl_of_care_c,pat_lv_of_care,event_type,pat_service
0,JCcd7ba7,131277369526,2019-10-31 10:46:00 UTC,4,Inpatient,1.0,66.0,Neonatal ICU - VC Only,Patient Update,Emergency Medicine
1,JCdc0a60,131281229519,2020-01-17 22:12:00 UTC,3,Inpatient,1.0,68.0,Newborn Nursery - VC Only,Patient Update,Emergency Medicine
2,JC2a0efc1,131280575648,2019-12-06 07:22:00 UTC,6,Inpatient,1.0,68.0,Newborn Nursery - VC Only,Patient Update,Emergency
3,JCdc5d49,131240161198,2017-10-08 06:59:00 UTC,2,Emergency Services,NaN,NaN,NaN,Census,Emergency
4,JCe56297,131079845274,2015-04-02 06:59:00 UTC,2,Emergency Services,NaN,NaN,NaN,Census,Emergency


In [190]:
# add the admit time column from the prediction results csv to the adt csv
adt_admit = adt.merge(cohort[['pat_enc_csn_id_coded', 'admit_time']], how='left', on=['pat_enc_csn_id_coded'])


adt_admit.columns
view_df(adt_admit[['pat_enc_csn_id_coded', 'effective_time_jittered_utc', 
                 'pat_class', 'admit_time']])

,pat_enc_csn_id_coded,effective_time_jittered_utc,pat_class,admit_time
0,131277369526,2019-10-31 10:46:00 UTC,Inpatient,2019-10-31 10:46:00+00:00
1,131281229519,2020-01-17 22:12:00 UTC,Inpatient,2020-01-17 22:12:00+00:00
2,131280575648,2019-12-06 07:22:00 UTC,Inpatient,2019-12-06 07:22:00+00:00
3,131240161198,2017-10-08 06:59:00 UTC,Emergency Services,2017-10-08 07:59:00+00:00
4,131079845274,2015-04-02 06:59:00 UTC,Emergency Services,2015-04-02 11:07:00+00:00


In [191]:
# mark the events with some labels that make things easier later
adt_admit = adt_admit.sort_values(['pat_enc_csn_id_coded', 'seq_num_in_enc'])

# **assumes dataframe is sorted by time

# mark whether previous entry had pat class Emergency Services
adt_admit['prev_emerg'] = adt_admit.pat_class.shift() == 'Emergency Services'
# mark whether current event has pat class Inpatient
adt_admit['curr_inpatient'] = adt_admit.pat_class == 'Inpatient'
# mark whether current event is continued csn as previous
adt_admit['continued'] = adt_admit.pat_enc_csn_id_coded.eq(
    adt_admit.pat_enc_csn_id_coded.shift())

# find our cases by taking the AND of these columns
cols = ['prev_emerg', 'curr_inpatient', 'continued']
adt_admit['first_ip'] = adt_admit[cols].all(axis=1)

view_df(adt_admit)

,anon_id,pat_enc_csn_id_coded,effective_time_jittered_utc,seq_num_in_enc,pat_class,base_pat_class_c,pat_lvl_of_care_c,pat_lv_of_care,event_type,pat_service,admit_time,prev_emerg,curr_inpatient,continued,first_ip
19178,JCe78a06,131062667066,2015-01-01 17:10:00 UTC,1,Emergency Services,3.0,NaN,NaN,Admission,Emergency,2015-01-02 01:01:00+00:00,False,False,False,False
19438,JCe78a06,131062667066,2015-01-02 00:32:00 UTC,2,Emergency Services,NaN,NaN,NaN,Transfer Out,Emergency,2015-01-02 01:01:00+00:00,True,False,True,False
19096,JCe78a06,131062667066,2015-01-02 00:32:00 UTC,3,Emergency Services,NaN,NaN,NaN,Transfer In,Emergency,2015-01-02 01:01:00+00:00,True,False,True,False
19434,JCe78a06,131062667066,2015-01-02 00:49:00 UTC,4,Emergency Services,NaN,NaN,NaN,Transfer Out,Emergency,2015-01-02 01:01:00+00:00,True,False,True,False
19100,JCe78a06,131062667066,2015-01-02 00:49:00 UTC,5,Emergency Services,NaN,NaN,NaN,Transfer In,Emergency,2015-01-02 01:01:00+00:00,True,False,True,False


# Look at first inpatient pat services

In [192]:
first_ips = adt_admit[adt_admit.first_ip == True]

print(first_ips.shape)
first_ips.pat_enc_csn_id_coded.nunique()

(44021, 15)


43980

In [193]:
# find first inpatient pat service for each csn
first_ip_pat_service = first_ips[['pat_enc_csn_id_coded', 'pat_service']].drop_duplicates()

print(first_ip_pat_service.shape)

(43989, 2)


# multiple emergency-->inpatient changes

Some CSNs have multiple emergency-->inpatient changes. This means they went from emergency-->inpatient-->emergency-->inpatient.

Take a look at some of these. The first change from emergency-->inpatient would be the admit time. Keep this occurrence. This only happen for 10 CSNs.

In [194]:
counts = first_ip_pat_service.groupby('pat_enc_csn_id_coded').count().sort_values('pat_service')

multi_csns = counts[counts.pat_service > 1]

print(multi_csns.shape)
multi_csns

(9, 1)


,pat_service
pat_enc_csn_id_coded,
131176788268,2
131171253065,2
131238854978,2
131244958373,2
131230527582,2
131201950621,2
131189499371,2
131230902129,2
131164129413,2


In [195]:
adt_admit[adt_admit.pat_enc_csn_id_coded == 131238854978].sort_values('seq_num_in_enc')[
    ['anon_id', 'pat_enc_csn_id_coded', 'effective_time_jittered_utc', 'admit_time',
       'seq_num_in_enc', 'pat_class', 'base_pat_class_c', 'pat_lvl_of_care_c',
       'pat_lv_of_care', 'event_type', 'pat_service'
       ]]

,anon_id,pat_enc_csn_id_coded,effective_time_jittered_utc,admit_time,seq_num_in_enc,pat_class,base_pat_class_c,pat_lvl_of_care_c,pat_lv_of_care,event_type,pat_service
4519,JCdb9b61,131238854978,2017-10-02 05:25:00 UTC,2017-10-02 07:56:00+00:00,1,Emergency Services,3.0,NaN,NaN,Admission,Emergency
5006,JCdb9b61,131238854978,2017-10-02 06:59:00 UTC,2017-10-02 07:56:00+00:00,2,Emergency Services,NaN,NaN,NaN,Census,Emergency
4581,JCdb9b61,131238854978,2017-10-02 07:45:00 UTC,2017-10-02 07:56:00+00:00,3,Emergency Services,NaN,NaN,NaN,Transfer Out,Emergency
5027,JCdb9b61,131238854978,2017-10-02 07:45:00 UTC,2017-10-02 07:56:00+00:00,4,Emergency Services,NaN,NaN,NaN,Transfer In,Emergency
307621,JCdb9b61,131238854978,2017-10-02 07:56:00 UTC,2017-10-02 07:56:00+00:00,5,Inpatient,1.0,5.0,Acute Care (Assessment or intervention q4-8),Patient Update,Emergency
305993,JCdb9b61,131238854978,2017-10-02 07:57:00 UTC,2017-10-02 07:56:00+00:00,6,Inpatient,NaN,5.0,Acute Care (Assessment or intervention q4-8),Transfer Out,Emergency
310885,JCdb9b61,131238854978,2017-10-02 07:57:00 UTC,2017-10-02 07:56:00+00:00,7,Emergency Services,NaN,5.0,Acute Care (Assessment or intervention q4-8),Transfer In,Emergency
311000,JCdb9b61,131238854978,2017-10-02 11:33:00 UTC,2017-10-02 07:56:00+00:00,8,Emergency Services,NaN,5.0,Acute Care (Assessment or intervention q4-8),Transfer Out,Emergency
307383,JCdb9b61,131238854978,2017-10-02 11:33:00 UTC,2017-10-02 07:56:00+00:00,9,Inpatient,NaN,5.0,Acute Care (Assessment or intervention q4-8),Transfer In,General Surgery
305544,JCdb9b61,131238854978,2017-10-03 01:29:00 UTC,2017-10-02 07:56:00+00:00,10,Inpatient,NaN,5.0,Acute Care (Assessment or intervention q4-8),Transfer Out,General Surgery


In [196]:
# keep the first occurrence of inpatient event
first_ips_keep = first_ips.sort_values(['pat_enc_csn_id_coded', 'seq_num_in_enc']).groupby('pat_enc_csn_id_coded').first().reset_index()

# now we have one event per person
print(first_ips_keep.pat_enc_csn_id_coded.nunique())
print(first_ips_keep.shape)

43980
(43980, 15)


In [197]:
view_df(first_ips_keep)

,pat_enc_csn_id_coded,anon_id,effective_time_jittered_utc,seq_num_in_enc,pat_class,base_pat_class_c,pat_lvl_of_care_c,pat_lv_of_care,event_type,pat_service,admit_time,prev_emerg,curr_inpatient,continued,first_ip
0,131062667066,JCe78a06,2015-01-02 01:01:00 UTC,7,Inpatient,1.0,6.0,Intermediate Care - With Cardiac Monitor,Transfer In,General Medicine (PAMF),2015-01-02 01:01:00+00:00,True,True,True,True
1,131062745090,JCd1c19e,2015-01-03 05:53:00 UTC,9,Inpatient,1.0,5.0,Acute Care (Assessment or intervention q4-8),Transfer In,Orthopaedic Surgery,2015-01-03 05:53:00+00:00,True,True,True,True
2,131062747648,JCd91eb2,2015-01-01 08:24:00 UTC,8,Inpatient,1.0,6.0,Intermediate Care - With Cardiac Monitor,Transfer In,Cardiology,2015-01-01 08:24:00+00:00,True,True,True,True
3,131062788358,JCe7cb4d,2015-01-01 23:39:00 UTC,3,Inpatient,1.0,5.0,Acute Care (Assessment or intervention q4-8),Transfer In,Psychiatry,2015-01-01 23:39:00+00:00,True,True,True,True
4,131063044001,JCe293de,2015-01-05 02:23:00 UTC,7,Inpatient,1.0,5.0,Acute Care (Assessment or intervention q4-8),Transfer In,Neurology,2015-01-05 02:23:00+00:00,True,True,True,True


In [198]:
first_ips_keep.columns

first_ips_save = first_ips_keep[['pat_enc_csn_id_coded', 'anon_id', 'admit_time',
                                 'effective_time_jittered_utc',
       'seq_num_in_enc', 'pat_class', 'base_pat_class_c', 'pat_lvl_of_care_c',
       'pat_lv_of_care', 'event_type', 'pat_service', ]]

view_df(first_ips_save)

,pat_enc_csn_id_coded,anon_id,admit_time,effective_time_jittered_utc,seq_num_in_enc,pat_class,base_pat_class_c,pat_lvl_of_care_c,pat_lv_of_care,event_type,pat_service
0,131062667066,JCe78a06,2015-01-02 01:01:00+00:00,2015-01-02 01:01:00 UTC,7,Inpatient,1.0,6.0,Intermediate Care - With Cardiac Monitor,Transfer In,General Medicine (PAMF)
1,131062745090,JCd1c19e,2015-01-03 05:53:00+00:00,2015-01-03 05:53:00 UTC,9,Inpatient,1.0,5.0,Acute Care (Assessment or intervention q4-8),Transfer In,Orthopaedic Surgery
2,131062747648,JCd91eb2,2015-01-01 08:24:00+00:00,2015-01-01 08:24:00 UTC,8,Inpatient,1.0,6.0,Intermediate Care - With Cardiac Monitor,Transfer In,Cardiology
3,131062788358,JCe7cb4d,2015-01-01 23:39:00+00:00,2015-01-01 23:39:00 UTC,3,Inpatient,1.0,5.0,Acute Care (Assessment or intervention q4-8),Transfer In,Psychiatry
4,131063044001,JCe293de,2015-01-05 02:23:00+00:00,2015-01-05 02:23:00 UTC,7,Inpatient,1.0,5.0,Acute Care (Assessment or intervention q4-8),Transfer In,Neurology


In [199]:
# save this file

savefile = savedir + "02_first_inpatient_event.csv"
first_ips_save.to_csv(savefile)

## Look at last Emergeny Services Pat Service

In [200]:
# mark the events with some labels that make things easier later

# **assumes dataframe is sorted by time

# mark whether next entry is inpatient
adt_admit['next_ip'] = adt_admit.pat_class.shift(-1) == 'Inpatient'
# mark whether current event has pat class Inpatient
adt_admit['curr_emerg'] = adt_admit.pat_class == 'Emergency Services'
# mark whether current event is continued csn as previous
adt_admit['not_continued'] = ~adt_admit.pat_enc_csn_id_coded.eq(
    adt_admit.pat_enc_csn_id_coded.shift(-1))

# find our cases by taking the AND of these columns
cols = ['next_ip', 'curr_emerg', 'continued']
adt_admit['last_emerg'] = adt_admit.curr_emerg & (adt_admit.next_ip | adt_admit.not_continued) 

adt_admit[['pat_enc_csn_id_coded', 'pat_class', 'last_emerg'] + cols].head(20)

,pat_enc_csn_id_coded,pat_class,last_emerg,next_ip,curr_emerg,continued
19178,131062667066,Emergency Services,False,False,True,False
19438,131062667066,Emergency Services,False,False,True,True
19096,131062667066,Emergency Services,False,False,True,True
19434,131062667066,Emergency Services,False,False,True,True
19100,131062667066,Emergency Services,False,False,True,True
18804,131062667066,Emergency Services,True,True,True,True
217107,131062667066,Inpatient,False,True,False,True
217366,131062667066,Inpatient,False,True,False,True
217842,131062667066,Inpatient,False,True,False,True
217409,131062667066,Inpatient,False,True,False,True


In [201]:
last_emergs = adt_admit[adt_admit.last_emerg == True]

# take the first one for each csn
last_emergs_keep = last_emergs.sort_values(['pat_enc_csn_id_coded', 'seq_num_in_enc']).groupby('pat_enc_csn_id_coded').first().reset_index()

# now we have one event per person
print(last_emergs_keep.pat_enc_csn_id_coded.nunique())
print(last_emergs_keep.shape)

view_df(last_emergs)

43980
(43980, 19)


,anon_id,pat_enc_csn_id_coded,effective_time_jittered_utc,seq_num_in_enc,pat_class,base_pat_class_c,pat_lvl_of_care_c,pat_lv_of_care,event_type,pat_service,admit_time,prev_emerg,curr_inpatient,continued,first_ip,next_ip,curr_emerg,not_continued,last_emerg
18804,JCe78a06,131062667066,2015-01-02 01:01:00 UTC,6,Emergency Services,NaN,NaN,NaN,Transfer Out,Emergency,2015-01-02 01:01:00+00:00,True,False,True,False,True,True,False,True
1208,JCd1c19e,131062745090,2015-01-03 05:53:00 UTC,8,Emergency Services,NaN,NaN,NaN,Transfer Out,Emergency,2015-01-03 05:53:00+00:00,True,False,True,False,True,True,False,True
77067,JCd91eb2,131062747648,2015-01-01 08:24:00 UTC,7,Emergency Services,NaN,NaN,NaN,Transfer Out,Emergency,2015-01-01 08:24:00+00:00,True,False,True,False,True,True,False,True
5398,JCe7cb4d,131062788358,2015-01-01 23:39:00 UTC,2,Emergency Services,NaN,NaN,NaN,Transfer Out,Emergency,2015-01-01 23:39:00+00:00,True,False,True,False,True,True,False,True
109842,JCe293de,131063044001,2015-01-05 02:23:00 UTC,6,Emergency Services,NaN,NaN,NaN,Transfer Out,Emergency,2015-01-05 02:23:00+00:00,True,False,True,False,True,True,False,True


In [202]:
last_emergs_keep.columns

last_emerg_save = last_emergs_keep[['pat_enc_csn_id_coded', 'anon_id', 'admit_time',
                                 'effective_time_jittered_utc',
       'seq_num_in_enc', 'pat_class', 'base_pat_class_c', 'pat_lvl_of_care_c',
       'pat_lv_of_care', 'event_type', 'pat_service', ]]

view_df(last_emerg_save)

,pat_enc_csn_id_coded,anon_id,admit_time,effective_time_jittered_utc,seq_num_in_enc,pat_class,base_pat_class_c,pat_lvl_of_care_c,pat_lv_of_care,event_type,pat_service
0,131062667066,JCe78a06,2015-01-02 01:01:00+00:00,2015-01-02 01:01:00 UTC,6,Emergency Services,NaN,NaN,None,Transfer Out,Emergency
1,131062745090,JCd1c19e,2015-01-03 05:53:00+00:00,2015-01-03 05:53:00 UTC,8,Emergency Services,NaN,NaN,None,Transfer Out,Emergency
2,131062747648,JCd91eb2,2015-01-01 08:24:00+00:00,2015-01-01 08:24:00 UTC,7,Emergency Services,NaN,NaN,None,Transfer Out,Emergency
3,131062788358,JCe7cb4d,2015-01-01 23:39:00+00:00,2015-01-01 23:39:00 UTC,2,Emergency Services,NaN,NaN,None,Transfer Out,Emergency
4,131063044001,JCe293de,2015-01-05 02:23:00+00:00,2015-01-05 02:23:00 UTC,6,Emergency Services,NaN,NaN,None,Transfer Out,Emergency


In [203]:
# save this file

savefile = savedir + "02_last_emerg_event.csv"
last_emerg_save.to_csv(savefile)

# Pat lv of care

Look at pat lv of care when patients are admitted

In [204]:
adt_admit.columns

Index(['anon_id', 'pat_enc_csn_id_coded', 'effective_time_jittered_utc',
       'seq_num_in_enc', 'pat_class', 'base_pat_class_c', 'pat_lvl_of_care_c',
       'pat_lv_of_care', 'event_type', 'pat_service', 'admit_time',
       'prev_emerg', 'curr_inpatient', 'continued', 'first_ip', 'next_ip',
       'curr_emerg', 'not_continued', 'last_emerg'],
      dtype='object')

In [205]:
# change admit time and effective time to datetime - these take a littl while to run!!

# change the effective time to datetime since read in from csv
adt_admit.effective_time_jittered_utc = pd.to_datetime(adt_admit.effective_time_jittered_utc)

# change admit time
adt_admit.admit_time = pd.to_datetime(adt_admit.admit_time, utc=True)

In [206]:
# filter down to 24hrs after admit
adt_admit['time_since_admit'] = adt_admit.effective_time_jittered_utc - adt_admit.admit_time

adt_results_24hr = adt_admit[(adt_admit.time_since_admit <= timedelta(hours=24)) &
                             (adt_admit.time_since_admit >= timedelta(hours=0))
                            ]

view_df(adt_results_24hr)

,anon_id,pat_enc_csn_id_coded,effective_time_jittered_utc,seq_num_in_enc,pat_class,base_pat_class_c,pat_lvl_of_care_c,pat_lv_of_care,event_type,pat_service,admit_time,prev_emerg,curr_inpatient,continued,first_ip,next_ip,curr_emerg,not_continued,last_emerg,time_since_admit
18804,JCe78a06,131062667066,2015-01-02 01:01:00+00:00,6,Emergency Services,NaN,NaN,NaN,Transfer Out,Emergency,2015-01-02 01:01:00+00:00,True,False,True,False,True,True,False,True,0 days 00:00:00
217107,JCe78a06,131062667066,2015-01-02 01:01:00+00:00,7,Inpatient,1.0,6.0,Intermediate Care - With Cardiac Monitor,Transfer In,General Medicine (PAMF),2015-01-02 01:01:00+00:00,True,True,True,True,True,False,False,False,0 days 00:00:00
217366,JCe78a06,131062667066,2015-01-02 07:59:00+00:00,8,Inpatient,NaN,6.0,Intermediate Care - With Cardiac Monitor,Census,General Medicine (PAMF),2015-01-02 01:01:00+00:00,False,True,True,False,True,False,False,False,0 days 06:58:00
1208,JCd1c19e,131062745090,2015-01-03 05:53:00+00:00,8,Emergency Services,NaN,NaN,NaN,Transfer Out,Emergency,2015-01-03 05:53:00+00:00,True,False,True,False,True,True,False,True,0 days 00:00:00
294639,JCd1c19e,131062745090,2015-01-03 05:53:00+00:00,9,Inpatient,1.0,5.0,Acute Care (Assessment or intervention q4-8),Transfer In,Orthopaedic Surgery,2015-01-03 05:53:00+00:00,True,True,True,True,True,False,False,False,0 days 00:00:00


In [207]:
# filter to inpatient events after admit time
ip_events = adt_results_24hr[(adt_results_24hr.admit_time <= adt_results_24hr.effective_time_jittered_utc) &
                     (adt_results_24hr.pat_class == 'Inpatient')]

ip_events = ip_events[ip_events.pat_enc_csn_id_coded.isin(cohort.pat_enc_csn_id_coded)]

ip_events.pat_enc_csn_id_coded.nunique()

43980

In [208]:
# keep only change of lv of care status

# sort by sequence 
ip_events = ip_events.sort_values(['pat_enc_csn_id_coded', 'seq_num_in_enc'])

# keep only those that change
change_care = ip_events[~(ip_events.pat_lv_of_care.eq(ip_events.pat_lv_of_care.shift()) &
                         ip_events.pat_enc_csn_id_coded.eq(ip_events.pat_enc_csn_id_coded.shift()))]

change_care.pat_enc_csn_id_coded.nunique()

43980

In [209]:
change_care.size

1059120

In [210]:
view_df(change_care[change_care.pat_enc_csn_id_coded.eq(change_care.shift().pat_enc_csn_id_coded)])

,anon_id,pat_enc_csn_id_coded,effective_time_jittered_utc,seq_num_in_enc,pat_class,base_pat_class_c,pat_lvl_of_care_c,pat_lv_of_care,event_type,pat_service,admit_time,prev_emerg,curr_inpatient,continued,first_ip,next_ip,curr_emerg,not_continued,last_emerg,time_since_admit
396473,JCda8edf,131063200606,2015-01-11 02:09:00+00:00,22,Inpatient,NaN,5.0,Acute Care (Assessment or intervention q4-8),Transfer In,Gynecology,2015-01-10 12:53:00+00:00,False,True,True,False,True,False,False,False,0 days 13:16:00
171671,JCd11b31,131063440327,2015-01-05 07:42:00+00:00,13,Inpatient,NaN,8.0,Critical Care,Transfer In,Critical Care,2015-01-04 08:12:00+00:00,False,True,True,False,True,False,False,False,0 days 23:30:00
260862,JCe3cf20,131064240713,2015-01-19 15:47:00+00:00,12,Inpatient,NaN,6.0,Intermediate Care - With Cardiac Monitor,Transfer In,General Medicine (University),2015-01-19 00:36:00+00:00,False,True,True,False,True,False,False,False,0 days 15:11:00
440822,JCe86314,131064254916,2015-01-06 23:48:00+00:00,16,Inpatient,NaN,5.0,Acute Care (Assessment or intervention q4-8),Transfer In,General Surgery,2015-01-06 02:34:00+00:00,False,True,True,False,True,False,False,False,0 days 21:14:00
143809,JCe06030,131064286661,2015-01-22 00:20:00+00:00,12,Inpatient,NaN,8.0,Critical Care,Transfer In,Cardiology,2015-01-21 00:36:00+00:00,False,True,True,False,True,False,False,False,0 days 23:44:00


In [211]:
# get the trajectory for each csn
trajectory = change_care[['pat_enc_csn_id_coded', 'pat_lv_of_care']].groupby('pat_enc_csn_id_coded')['pat_lv_of_care'].apply(list).reset_index(name='trajectory')

view_df(trajectory)

,pat_enc_csn_id_coded,trajectory
0,131062667066,[Intermediate Care - With Cardiac Monitor]
1,131062745090,[Acute Care (Assessment or intervention q4-8)]
2,131062747648,[Intermediate Care - With Cardiac Monitor]
3,131062788358,[Acute Care (Assessment or intervention q4-8)]
4,131063044001,[Acute Care (Assessment or intervention q4-8)]


In [212]:
# add a trajectory string column and also trajectory count
trajectory['trajectory_string'] = [' -> '.join(map(str, l)) for l in trajectory['trajectory']]
trajectory['trajectory_length'] = trajectory['trajectory'].str.len()

In [213]:
view_df(trajectory[trajectory.trajectory_length > 1])

,pat_enc_csn_id_coded,trajectory,trajectory_string,trajectory_length
12,131063200606,"[Critical Care, Acute Care (Assessment or inte...",Critical Care -> Acute Care (Assessment or int...,2
24,131063440327,"[Intermediate Care - With Cardiac Monitor, Cri...",Intermediate Care - With Cardiac Monitor -> Cr...,2
87,131064240713,"[Acute Care (Assessment or intervention q4-8),...",Acute Care (Assessment or intervention q4-8) -...,2
101,131064254916,"[Critical Care, Acute Care (Assessment or inte...",Critical Care -> Acute Care (Assessment or int...,2
108,131064286661,"[Intermediate Care - With Cardiac Monitor, Cri...",Intermediate Care - With Cardiac Monitor -> Cr...,2


In [214]:
# save this file
savefile = savedir + "02_trajectory.csv"
trajectory.to_csv(savefile)

# Combine the data into one dataframe

In [215]:
# format the first inpatient event
first_ip_less = first_ips_save[['anon_id', 'pat_enc_csn_id_coded',
       'pat_lv_of_care', 'event_type', 'pat_service']]

first_ip_less.rename({
                      'pat_service':'first_ip_pat_service',
                      'event_type':'first_ip_event_type',
                      'pat_lv_of_care':'first_ip_lv_of_care'
                     },
                    inplace=True, axis=1)

view_df(first_ip_less)

/home/eulalio/.local/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,anon_id,pat_enc_csn_id_coded,first_ip_lv_of_care,first_ip_event_type,first_ip_pat_service
0,JCe78a06,131062667066,Intermediate Care - With Cardiac Monitor,Transfer In,General Medicine (PAMF)
1,JCd1c19e,131062745090,Acute Care (Assessment or intervention q4-8),Transfer In,Orthopaedic Surgery
2,JCd91eb2,131062747648,Intermediate Care - With Cardiac Monitor,Transfer In,Cardiology
3,JCe7cb4d,131062788358,Acute Care (Assessment or intervention q4-8),Transfer In,Psychiatry
4,JCe293de,131063044001,Acute Care (Assessment or intervention q4-8),Transfer In,Neurology


In [216]:
# format the last emerg event
last_emerg_less = last_emerg_save[['anon_id', 'pat_enc_csn_id_coded',
       'pat_lv_of_care', 'event_type', 'pat_service']]

last_emerg_less.rename({
                      'pat_service':'last_emerg_pat_service',
                      'event_type':'last_emerg_event_type',
                      'pat_lv_of_care':'last_emerg_lv_of_care'
                     },
                    inplace=True, axis=1)

view_df(last_emerg_less)

/home/eulalio/.local/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,anon_id,pat_enc_csn_id_coded,last_emerg_lv_of_care,last_emerg_event_type,last_emerg_pat_service
0,JCe78a06,131062667066,None,Transfer Out,Emergency
1,JCd1c19e,131062745090,None,Transfer Out,Emergency
2,JCd91eb2,131062747648,None,Transfer Out,Emergency
3,JCe7cb4d,131062788358,None,Transfer Out,Emergency
4,JCe293de,131063044001,None,Transfer Out,Emergency


In [217]:
trajectory_less = trajectory[['pat_enc_csn_id_coded', 'trajectory_string', 'trajectory_length']]

trajectory_less.rename({'trajectory_string':'trajectory'},
                    inplace=True, axis=1)

view_df(trajectory_less)

/home/eulalio/.local/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,pat_enc_csn_id_coded,trajectory,trajectory_length
0,131062667066,Intermediate Care - With Cardiac Monitor,1
1,131062745090,Acute Care (Assessment or intervention q4-8),1
2,131062747648,Intermediate Care - With Cardiac Monitor,1
3,131062788358,Acute Care (Assessment or intervention q4-8),1
4,131063044001,Acute Care (Assessment or intervention q4-8),1


In [218]:
ip_emerg = first_ip_less.merge(last_emerg_less, how='outer')
view_df(ip_emerg)

,anon_id,pat_enc_csn_id_coded,first_ip_lv_of_care,first_ip_event_type,first_ip_pat_service,last_emerg_lv_of_care,last_emerg_event_type,last_emerg_pat_service
0,JCe78a06,131062667066,Intermediate Care - With Cardiac Monitor,Transfer In,General Medicine (PAMF),None,Transfer Out,Emergency
1,JCd1c19e,131062745090,Acute Care (Assessment or intervention q4-8),Transfer In,Orthopaedic Surgery,None,Transfer Out,Emergency
2,JCd91eb2,131062747648,Intermediate Care - With Cardiac Monitor,Transfer In,Cardiology,None,Transfer Out,Emergency
3,JCe7cb4d,131062788358,Acute Care (Assessment or intervention q4-8),Transfer In,Psychiatry,None,Transfer Out,Emergency
4,JCe293de,131063044001,Acute Care (Assessment or intervention q4-8),Transfer In,Neurology,None,Transfer Out,Emergency


In [219]:
ip_emerg_traj = ip_emerg.merge(trajectory_less, how='outer')
view_df(ip_emerg_traj)

,anon_id,pat_enc_csn_id_coded,first_ip_lv_of_care,first_ip_event_type,first_ip_pat_service,last_emerg_lv_of_care,last_emerg_event_type,last_emerg_pat_service,trajectory,trajectory_length
0,JCe78a06,131062667066,Intermediate Care - With Cardiac Monitor,Transfer In,General Medicine (PAMF),None,Transfer Out,Emergency,Intermediate Care - With Cardiac Monitor,1
1,JCd1c19e,131062745090,Acute Care (Assessment or intervention q4-8),Transfer In,Orthopaedic Surgery,None,Transfer Out,Emergency,Acute Care (Assessment or intervention q4-8),1
2,JCd91eb2,131062747648,Intermediate Care - With Cardiac Monitor,Transfer In,Cardiology,None,Transfer Out,Emergency,Intermediate Care - With Cardiac Monitor,1
3,JCe7cb4d,131062788358,Acute Care (Assessment or intervention q4-8),Transfer In,Psychiatry,None,Transfer Out,Emergency,Acute Care (Assessment or intervention q4-8),1
4,JCe293de,131063044001,Acute Care (Assessment or intervention q4-8),Transfer In,Neurology,None,Transfer Out,Emergency,Acute Care (Assessment or intervention q4-8),1


In [220]:
# save this combined df
savefile = savedir + "02_combined_ip_emerg_traj.csv"
ip_emerg_traj.to_csv(savefile)

# Decription of columns

- first_ip_* = labels for the first inpatient event (admit time)
- last_emerg_* = labels for the last emergency event (just before admit time)
- trajectory = CSNs movement through care levels from admit time to 24 hours after admit
- trajectory_length = number of times CSN moves around in trajectory